In [1]:
%run imports_and_functions.ipynb

from dbfread import DBF

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Creating Final Businesses Dataframe within City Radius

In [72]:
df = pd.read_csv('CharlotteBusinessesUpdated.csv', sep=';', compression='gzip')
df.info()
# NOT more businesses in Yelp for Charlotte for 2019 versus 2014 Kaggle dataset, will use older set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6630 entries, 0 to 6629
Data columns (total 13 columns):
Unnamed: 0       6630 non-null int64
alias            6630 non-null object
attributes       0 non-null float64
categories       6630 non-null object
coordinates      6630 non-null object
distance         6630 non-null float64
latitude         6588 non-null float64
longitude        6588 non-null float64
main_category    6630 non-null object
name             6630 non-null object
price            2553 non-null object
rating           6630 non-null float64
review_count     6630 non-null int64
dtypes: float64(5), int64(2), object(6)
memory usage: 673.4+ KB


In [20]:
df = pd.read_csv('CharlotteBusinesses.csv', sep=';', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9204 entries, 0 to 9203
Data columns (total 12 columns):
address         8851 non-null object
attributes      7996 non-null object
categories      9179 non-null object
city            9204 non-null object
latitude        9204 non-null float64
longitude       9204 non-null float64
name            9204 non-null object
neighborhood    6266 non-null object
postal_code     9166 non-null float64
review_count    9204 non-null int64
stars           9204 non-null float64
state           9204 non-null object
dtypes: float64(4), int64(1), object(7)
memory usage: 863.0+ KB


In [73]:
def dropduplicate_yelp_entries(df) :
    dups = df[df[['latitude','longitude','alias']].duplicated(keep=False)].sort_values('latitude')
    mask1 = dups[['latitude','longitude','alias']].duplicated(keep='first')
    mask2 = dups[['latitude','longitude','alias']].duplicated(keep=False)
    mask = mask2 & ~mask1
#     print(dups)
    for index in dups[mask].index :
        dupset = dups[(dups['alias']==dups.loc[index,'alias']) & (dups['longitude']==dups.loc[index,'longitude'])].sort_values(
            'review_count', ascending=False)
        dropindex = dupset.iloc[1:,0].index
        df.drop(dropindex, axis=0, inplace=True)
        
def getPriceRange(string) :
    regex = re.compile("RestaurantsPriceRange2\\': \\'(\d)")
    try :
        match = regex.search(string)
        return int(match.groups(0)[0])
    except : return 0
    
def convertpricecolumn(pricesymbol) :
    try : num = len(pricesymbol)
    except : num = 0
    return num

def get_main_category(catstring) :
    string = ''
    try :
        for cat in main_categories :
            if cat in catstring : string = string + cat + ', '
    except : pass
    return string[0:(len(string)-2)]

In [74]:
print(df.shape)
dropduplicate_yelp_entries(df)
df.shape

(6630, 13)


(5380, 13)

In [75]:
df['price'] = df['price'].map(convertpricecolumn)
df['price'].value_counts()

0    3436
2    1083
1     644
3     184
4      33
Name: price, dtype: int64

In [60]:
# df['main_category'] = df['categories'].map(get_main_category)
df['main_category'].value_counts()

restaurants           647
homeservices          523
shopping              516
health                462
food                  457
beautysvc             451
localservices         332
automotive            317
eventservices         229
professional          218
active                218
nightlife             201
financialservices     143
arts                  133
realestate            125
hotelstravel          110
pets                   98
education              76
religiousorgs          49
publicservicesgovt     46
massmedia              29
Name: main_category, dtype: int64

In [36]:
# SET FOR EACH CITY: maxradius, testlat, testlong
maxradius = 7000
testlat = 35.225327
testlong = -80.840803 # for Charlotte, NC

In [76]:
df['distance'] = calcDistances(testlat, testlong, df)

df[df['distance']<maxradius].shape

(4961, 13)

In [63]:
df['distance'].sort_values()

3600       11.550093
5180       37.090888
2900       42.917939
250        52.772527
4180       56.854912
2400       66.441148
1400       70.723503
251        84.916799
5183       89.950993
1850       89.950993
3602       90.089401
5182       90.221564
5184       90.221564
5186       90.460901
4182      105.572795
5188      105.572795
252       105.572795
1851      105.599473
1853      109.104641
3603      109.488477
5196      110.039098
5195      110.039098
5981      110.039098
5194      110.039098
5193      110.039098
254       110.039098
1404      110.039098
1405      110.039098
4189      110.039098
1700      110.039098
1854      110.039098
1856      110.039098
3604      110.039098
4150      110.083960
2401      110.083960
4191      110.966939
4185      111.659071
5199      120.936866
0         122.395298
5197      124.605634
1702      125.137547
5200      125.995946
5198      130.139714
5982      130.735603
1         130.979824
4186      132.412080
5190      132.933751
5202      133

In [12]:
df[df[['latitude','longitude']].duplicated(keep=False)].sort_values('latitude')

address  \
lat_index long_index                                                     
35.021    -80.845996                              11611 Ardrey Kell Rd   
          -80.845996                            11611 Ardrey Kell Road   
35.023    -80.846807                             16646 Hawfield Way Dr   
          -80.846807                             16646 Hawfield Way Dr   
          -80.847231                             16640 Hawfield Way Dr   
          -80.847231                             16640 Hawfield Way Dr   
35.024    -80.848949                      16615 Lancaster Hwy, Ste 104   
          -80.848949                      16615 Lancaster Hwy, Ste 106   
          -80.848949                16615 Lancaster Highway, Suite 107   
          -80.848949                      16615 Lancaster Hwy, Ste 105   
35.027    -80.838774                           16709 Orchard Stone Run   
          -80.838774                           16709 Orchard Stone Run   
          -80.838774                  16709 Orchard Stone Run, Ste 320   
          -80.838774                           16709 Orchard Stone Run   
35.028    -80.850945                      16139 Lancaster Hwy, Ste 100   
          -80.850945                               16139 Lancaster Hwy   
          -80.851019                               16131 Lancaster Hwy   
          -80.851019                      16131 Lancaster Hwy, Ste 110   
35.029    -80.851159                               16131 Lancaster Hwy   
          -80.851159                        16131 Lancaster Hwy, Ste 3   
          -80.851159                        16131 Lancaster Hwy, Ste 4   
          -80.850153                          16049 Johnston Rd, Ste J   
          -80.850153                               16049-K Johnston Rd   
          -80.850153                          16049 Johnston Rd, Ste H   
          -80.850153                                 16049 Johnston Rd   
          -80.850153                          16049 Johnston Rd, Ste I   
35.030    -80.853552                        16015 Lancaster Hwy, Ste D   
          -80.853552                  16015 Lancaster Highway, Suite C   
35.033    -80.806544                             5933 Blakeney Park Dr   
          -80.806544                    5933 Blakeney Park Dr, Ste 100   
          -80.808061            8912 Blakeney Professional Dr, Ste 100   
          -80.808061             8912 Blakeney Professional Dr, Ste A2   
          -80.806856                             5815 Blakeney Park Dr   
          -80.806856                             5815 Blakeney Park Dr   
35.034    -80.804713                              9935 Rea Road, Ste C   
          -80.804713                                9935 Rea Rd, Ste B   
          -80.804713                                       9935 Rea Rd   
          -80.806346                                       9824 Rea Rd   
          -80.806346                                9824 Rea Rd, Ste B   
35.035    -80.804514                                       9831 Rea Rd   
          -80.804514                                       9831 Rea Rd   
          -80.804514                                9831 Rea Rd, Ste B   
          -80.807239                              9852 Rea Rd, Ste A-1   
          -80.807239                               9852 Rea Rd, Ste F2   
          -80.807369                               9852 Rea Rd, Ste D2   
          -80.807369                                       9852 Rea Rd   
          -80.807369                               9852 Rea Rd, Ste A2   
          -80.806120                                9816 Rea Rd, Ste I   
          -80.806120                                       9816 Rea Rd   
          -80.806120                                       9816 Rea Rd   
35.036    -80.804473                                       9815 Rea Rd   
          -80.804473                                       9815 Rae Rd   
          -80.809316                                9876 Rea Rd, St

In [77]:
rowstodrop = df[(df['distance']>maxradius) | df['latitude'].isnull() | df['longitude'].isnull()].index

df.drop(rowstodrop, axis=0, inplace=True)
df.shape

(4961, 13)

In [78]:
df['lat_index'] = round(df['latitude'], 3)
df['long_index'] = df['longitude']

df = df.set_index(['lat_index','long_index'], drop=False).sort_index()
df = df.sort_index(level = [0,1])

In [66]:
df.head()

Unnamed: 0                              alias  \
lat_index long_index                                                  
35.163    -80.849807         796  charlotte-car-transport-charlotte   
          -80.849670        4566       aaa-pet-services-charlotte-3   
          -80.849541        1670              paint-craze-charlotte   
          -80.848420        3550          dy-locksmith-charlotte-13   
          -80.848419        3551         cielo-apartments-charlotte   

                      attributes  \
lat_index long_index               
35.163    -80.849807         NaN   
          -80.849670         NaN   
          -80.849541         NaN   
          -80.848420         NaN   
          -80.848419         NaN   

                                                             categories  \
lat_index long_index                                                      
35.163    -80.849807  [{'alias': 'vehicleshipping', 'title': 'Vehicl...   
          -80.849670  [{'alias': 'vet', 'title': 'Veterinarians'}, {...   
          -80.849541  [{'alias': 'artschools', 'title': 'Art Schools...   
          -80.848420  [{'alias': 'locksmiths', 'title': 'Keys & Lock...   
          -80.848419   [{'alias': 'apartments', 'title': 'Apartments'}]   

                                                            coordinates  \
lat_index long_index                                                      
35.163    -80.849807    {'latitude': 35.16333, 'longitude': -80.849807}   
          -80.849670     {'latitude': 35.16303, 'longitude': -80.84967}   
          -80.849541  {'latitude': 35.1628304067365, 'longitude': -8...   
          -80.848420     {'latitude': 35.16336, 'longitude': -80.84842}   
          -80.848419   {'latitude': 35.163356, 'longitude': -80.848419}   

                         distance   latitude  longitude  main_category  \
lat_index long_index                                                     
35.163    -80.849807  6926.911525  35.163330 -80.849807     automotive   
          -80.849670  6958.502877  35.163030 -80.849670           pets   
          -80.849541  6979.144871  35.162830 -80.849541  eventservices   
          -80.848420  6909.784556  35.163360 -80.848420   homeservices   
          -80.848419  6910.216948  35.163356 -80.848419   homeservices   

                                         name price  rating  review_count  \
lat_index long_index                                                        
35.163    -80.849807  Charlotte Car Transport   NaN     4.0             3   
          -80.849670         AAA Pet Services   NaN     4.0            10   
          -80.849541              Paint Craze   NaN     5.0             8   
          -80.848420             DY Locksmith   NaN     5.0             6   
          -80.848419         Cielo Apartments   NaN     3.5            11   

                      lat_index  long_index  
lat_index long_index                         
35.163    -80.849807     35.163  -80.849807  
          -80.849670     35.163  -80.849670  
          -80.849541     35.163  -80.849541  
          -80.848420     35.163  -80.848420  
          -80.848419     35.163  -80.848419

In [67]:
cat_alias_dict = json.load(open('cat_alias_dict.json','r'))

def categories_to_alias2(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    rawlis = list(set(catstring.split(', ')))
    lis = []
    for item in rawlis :
        if 'alias' in item :
            alias = item.split(':')[1].strip().strip("'")
            lis += [alias]
#     print(lis)
    lis = list(map(get_alias2,lis))
    lis = [item for item in lis if item != '_other']
    lis += ['_' + dfrow['main_category']]
    # some new categories are not in category dictionary, and each row already has main category
    # so am taking out all instances where business category could not be found
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias2(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat_alias_dict[cat])])
    except : alias = '_other'
    return alias

def categories_to_alias(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    lis = list(set(catstring.split(', ')))
    lis = map(get_alias,lis)
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat)])
    except : alias = '_other'
    return alias

## Function to map titles to macro-categories if not in top 150

dict_2A = json.load(open('dict_2A.json','r'))
dict_2B = json.load(open('dict_2B.json','r'))

def map_category(category):
    if category not in top_148:
        if category in dict_2B.keys():
            title = dict_2B[category]
        else:
            title = category
        if title in dict_2A.keys():
            title_2 = dict_2A[title]
            return(title_2)
        else:
            return(title)
    else:
        return(category)

In [79]:
df['alias_cats'] = df.apply(categories_to_alias2, axis=1)

In [81]:

df.to_csv('CharlotteBusinessesFinalUpdated.csv',sep=';',header=True,index=True,compression='gzip')

In [80]:
df['alias_cats']

lat_index  long_index
35.163     -80.849807                                  0_auto 0_automotive
           -80.849670                               0_groomer 0_vet 0_pets
           -80.849541          0_eventservices 0_venues 0_education 0_arts
           -80.848420                 0_auto 0_homeservices 0_professional
           -80.848419                          0_homeservices 0_apartments
35.164     -80.850464                            2_beautysvc 2_othersalons
           -80.850463                             0_health 0_chiropractors
           -80.850463                           2_localservices 2_shopping
           -80.850460            2_medicalspa 2_waxing 2_skincare 2_health
           -80.850398                           2_hairstylists 2_beautysvc
           -80.850360                           0_hairstylists 0_beautysvc
           -80.850324                            0_healthtrainers 0_active
           -80.850100                           3_beautysvc 3_hairstylists
   

### Census Block Population and Income to X_train Coordinates DataFrame

In [2]:
df_pop = pd.read_csv('tabblock2010_37_pophu.csv')
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288987 entries, 0 to 288986
Data columns (total 3 columns):
BLOCKCE,C,4       288987 non-null int64
BLOCKID10,C,15    288987 non-null int64
POP10,N,9,0       288987 non-null int64
dtypes: int64(3)
memory usage: 6.6 MB


In [53]:
table = DBF('tl_2014_37_tabblock10.dbf')
type(table)

dbfread.dbf.DBF

In [54]:
df_coords = pd.DataFrame(iter(table))
df_coords.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10
0,37,037,020402,1014,370370204021014,Block 1014,G5040,U,81955,C,S,1834383,8537,+35.7507675,-079.4653092
1,37,037,020800,1031,370370208001031,Block 1031,G5040,R,,,S,1207684,4076,+35.7376079,-079.1952368
2,37,037,020702,1055,370370207021055,Block 1055,G5040,R,,,S,2316525,30428,+35.7324096,-078.9631283
3,37,037,020200,2044,370370202002044,Block 2044,G5040,R,,,S,873908,7445,+35.7459976,-079.3855329
4,37,037,020200,1004,370370202001004,Block 1004,G5040,R,,,S,5128921,49040,+35.8293086,-079.4694158


In [55]:
df_coords = df_coords[['GEOID10','INTPTLAT10','INTPTLON10','UR10','ALAND10','AWATER10']]
df_coords.to_csv('CharlotteCoords.csv',sep=';',header=True,index=True,compression='gzip')

In [56]:
def cut(geoid) :
    index = str(geoid)
    return int(index[0:(len(index)-3)])

def matchincome(geoid) :
    try : return blockgroupincome.loc[geoid,'income']
    except : return 0

def matchpop(geoid) :
    try : return blockgrouppop.loc[geoid,'population']
    except : return 0

In [57]:
populations = pd.read_csv('tabblock2010_37_pophu.csv')
populations.head()

,"BLOCKCE,C,4","BLOCKID10,C,15","POP10,N,9,0"
0,2010,370010218012010,0
1,2013,370010218012013,0
2,2009,370010218012009,0
3,2001,370010218012001,0
4,3007,370010212063007,0


In [58]:
populations.columns = ['block','GEO.id2','population']
populations['blockgroup'] = populations['GEO.id2'].map(cut)
populations.set_index(['GEO.id2'],inplace=True,drop=False)
populations.head()

,block,GEO.id2,population,blockgroup
GEO.id2,,,,
370010218012010,2010,370010218012010,0,370010218012
370010218012013,2013,370010218012013,0,370010218012
370010218012009,2009,370010218012009,0,370010218012
370010218012001,2001,370010218012001,0,370010218012
370010212063007,3007,370010212063007,0,370010212063


In [59]:
blockgroupincome = pd.read_csv('Charlotte.csv', usecols=[2,4,6])
blockgroupincome.head()

,GEO.id2,HD01_VD01,income
0,Id2,Estimate; Total,Estimate; Per capita income in the past 12 mon...
1,371190001001,1073,107319
2,371190001002,1945,89079
3,371190001003,901,145815
4,371190001004,173,45431


In [60]:
blockgroupincome.drop(0, axis=0, inplace=True)
blockgroupincome.columns = ['GEO.id2','population','income']
blockgroupincome['GEO.id2'] = blockgroupincome['GEO.id2'].map(int)
blockgroupincome.set_index('GEO.id2', inplace=True)
blockgroupincome.head()

,population,income
GEO.id2,,
371190001001,1073,107319
371190001002,1945,89079
371190001003,901,145815
371190001004,173,45431
371190001005,569,71041


In [61]:
blockgroupincome.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555 entries, 371190001001 to 371199803001
Data columns (total 2 columns):
population    555 non-null object
income        555 non-null object
dtypes: object(2)
memory usage: 13.0+ KB


In [63]:
populations['income'] = populations['blockgroup'].map(matchincome)
populations['income'].value_counts()

0         276391
-            117
41898        112
27918        102
66949        101
45799         97
44615         97
35424         91
23222         84
41477         83
37101         80
47022         72
41433         71
41402         70
13399         68
23815         67
35151         66
58452         64
32582         64
18606         63
44155         62
29035         62
26019         61
21547         60
18558         58
27799         58
28069         56
19870         55
39668         55
67189         54
35706         54
36003         53
36432         53
64743         53
28387         52
28452         52
74896         50
12976         50
47511         50
41987         50
30921         48
42937         47
38280         47
12179         47
79555         47
64955         47
44885         46
54627         45
59258         45
32824         45
25419         44
26694         44
22577         44
49368         43
17175         43
19079         43
26630         42
28812         42
13149         

In [14]:
blockgrouppop = blockgroupincome

populations['blockgrouppop'] = populations['blockgroup'].map(matchpop)
populations.tail()

,block,GEO.id2,population,blockgroup,income,blockgrouppop
GEO.id2,,,,,,
371999602002036,2036,371999602002036,25,371999602002,0,0
371999601011019,1019,371999601011019,0,371999601011,0,0
371999603002058,2058,371999603002058,0,371999603002,0,0
371999602004025,4025,371999602004025,0,371999602004,0,0
371999602001001,1001,371999602001001,0,371999602001,0,0


In [64]:
populations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288987 entries, 370010218012010 to 371999602001001
Data columns (total 5 columns):
block         288987 non-null int64
GEO.id2       288987 non-null int64
population    288987 non-null int64
blockgroup    288987 non-null int64
income        288987 non-null object
dtypes: int64(4), object(1)
memory usage: 13.2+ MB


### X_train Coordinates DataFrame

In [65]:
features = ['distance','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea','block']

In [66]:
def blocksdropna(df) :
    df.drop(df[df['income']==0].index, inplace=True)
    df.drop(df[df['income']=='-'].index, inplace=True)
    df.drop(df[df['population']<20].index, inplace=True)  # min 20 people for block to be counted

In [67]:
blocks = pd.read_csv('CharlotteCoords.csv',sep=';',compression='gzip')
blocks.head()

,Unnamed: 0,GEOID10,INTPTLAT10,INTPTLON10,UR10,ALAND10,AWATER10
0,0,370370204021014,35.750768,-79.465309,U,1834383,8537
1,1,370370208001031,35.737608,-79.195237,R,1207684,4076
2,2,370370207021055,35.732410,-78.963128,R,2316525,30428
3,3,370370202002044,35.745998,-79.385533,R,873908,7445
4,4,370370202001004,35.829309,-79.469416,R,5128921,49040


In [68]:
blocks.drop('Unnamed: 0', axis=1, inplace=True)
blocks.columns = ['GEO.id2','latitude','longitude','urban','landarea','waterarea']
blocks['blockgroup'] = blocks['GEO.id2'].map(cut)
blocks['GEO.id2'] = blocks['GEO.id2'].map(int)
blocks = blocks.set_index('GEO.id2')
blocks.sort_index(inplace=True)
blocks.head()

,latitude,longitude,urban,landarea,waterarea,blockgroup
GEO.id2,,,,,,
370010201001000,36.101612,-79.445334,U,29235,0,370010201001
370010201001001,36.101822,-79.447900,U,24328,0,370010201001
370010201001002,36.102302,-79.449814,U,14105,0,370010201001
370010201001003,36.102773,-79.451870,U,21198,0,370010201001
370010201001004,36.101950,-79.452455,U,31825,0,370010201001


In [69]:
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288987 entries, 370010201001000 to 371999604003117
Data columns (total 6 columns):
latitude      288987 non-null float64
longitude     288987 non-null float64
urban         288987 non-null object
landarea      288987 non-null int64
waterarea     288987 non-null int64
blockgroup    288987 non-null int64
dtypes: float64(2), int64(3), object(1)
memory usage: 15.4+ MB


In [72]:
blocks = blocks.join(populations[['income','population']], how='inner')
blocks.shape

(288987, 8)

In [73]:
# blocks['income'].sort_values(ascending=False)
blocks['income'].value_counts()

0         276391
-            117
41898        112
27918        102
66949        101
44615         97
45799         97
35424         91
23222         84
41477         83
37101         80
47022         72
41433         71
41402         70
13399         68
23815         67
35151         66
32582         64
58452         64
18606         63
29035         62
44155         62
26019         61
21547         60
18558         58
27799         58
28069         56
19870         55
39668         55
67189         54
35706         54
64743         53
36003         53
36432         53
28387         52
28452         52
47511         50
12976         50
41987         50
74896         50
30921         48
79555         47
64955         47
38280         47
12179         47
42937         47
44885         46
32824         45
59258         45
54627         45
22577         44
25419         44
26694         44
19079         43
49368         43
17175         43
26630         42
13149         42
28812         

In [74]:
blocksdropna(blocks)
blocks.shape

(7259, 8)

In [34]:
blocks[(blocks['population']!=0) & (blocks['income']==0)] # must drop as don't have y target (income)

,latitude,longitude,urban,landarea,waterarea,blockgroup,income,blockgrouppop,population
GEO.id2,,,,,,,,,


In [49]:
testlat, testlong  # used as center of Charlotte

(35.225327, -80.840803)

In [75]:
blocks['distance_from_city'] = blocks.apply(
    (lambda row : calcDist(row['latitude'],row['longitude'],testlat, testlong)),         
    axis=1)

In [76]:
blocks[blocks['distance_from_city']<6000].shape
# blocks.shape

(1436, 9)

In [52]:
blocks = blocks.reindex(['distance_from_city','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea'], axis=1)
blocks.head()

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
GEO.id2,,,,,,,,,,
371190001001007,608.105314,U,+35.2287927,-080.8459781,371190001001,107319,1073,186,9488,0
371190001001017,430.509468,U,+35.2264923,-080.8453137,371190001001,107319,1073,129,18829,0
371190001002001,920.691373,U,+35.2323789,-080.8461346,371190001002,89079,1945,204,22452,0
371190001002003,928.750975,U,+35.2330826,-080.8446433,371190001002,89079,1945,260,12313,0
371190001002004,803.735637,U,+35.2321399,-080.8438048,371190001002,89079,1945,138,17070,0


In [148]:
blocks = blocks.reindex(['distance_from_city','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea'], axis=1)
blocks.head()

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
GEO.id2,,,,,,,,,,
371190001001000,907.183167,U,35.230750,-80.848262,371190001001,107319,1073,0,24797,0
371190001001001,731.862622,U,35.229594,-80.846934,371190001001,107319,1073,0,25252,0
371190001001002,759.811243,U,35.228813,-80.847987,371190001001,107319,1073,0,21098,0
371190001001003,847.322318,U,35.228983,-80.848975,371190001001,107319,1073,0,707,0
371190001001004,809.529148,U,35.228088,-80.849034,371190001001,107319,1073,0,21202,0


In [149]:
blocks[blocks['distance_from_city']<6000][['population']].sum()

population    127258
dtype: int64

#### Wacky Error with Block Rows being half what they should be, Reloading from previous csv

In [2]:
blocks = pd.read_csv('CharlotteTrainRowCoords.csv',
          sep=';', compression='gzip', index_col='Unnamed: 0')
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2739 entries, 371190001001000 to 1371190053062011
Data columns (total 10 columns):
distance_from_city    2739 non-null float64
urban                 2739 non-null object
latitude              2739 non-null float64
longitude             2739 non-null float64
blockgroup            2739 non-null int64
income                2739 non-null object
blockgrouppop         2739 non-null int64
population            2739 non-null int64
landarea              2739 non-null int64
waterarea             2739 non-null int64
dtypes: float64(3), int64(5), object(2)
memory usage: 235.4+ KB


In [3]:
blocks.duplicated().sum()

171

In [4]:
blocks = blocks.drop_duplicates(keep='first')
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2568 entries, 371190001001000 to 371199803001015
Data columns (total 10 columns):
distance_from_city    2568 non-null float64
urban                 2568 non-null object
latitude              2568 non-null float64
longitude             2568 non-null float64
blockgroup            2568 non-null int64
income                2568 non-null object
blockgrouppop         2568 non-null int64
population            2568 non-null int64
landarea              2568 non-null int64
waterarea             2568 non-null int64
dtypes: float64(3), int64(5), object(2)
memory usage: 220.7+ KB


In [5]:
def bootstrapBlocks(blocksdf) :
    df = blocksdf.copy()
    print(df.index)
    for index in blocksdf[blocksdf['population']>100].index :
        iterpop = int(np.ceil(blocksdf.loc[index,'population'] / 100))
        lis = [pd.Series(data=blocksdf.loc[index].values, name=index+(10**15*i),
                        index=blocksdf.columns) for i in range(1,iterpop)]
#         print(len(lis))
        df = df.append(lis) # more efficient and faster to append all at once
    return df

In [6]:
blocks = bootstrapBlocks(blocks)
blocks.shape

Int64Index([371190001001000, 371190001001001, 371190001001002, 371190001001003,
            371190001001004, 371190001001005, 371190001001006, 371190001001007,
            371190001001008, 371190001001009,
            ...
            371199803001006, 371199803001007, 371199803001008, 371199803001009,
            371199803001010, 371199803001011, 371199803001012, 371199803001013,
            371199803001014, 371199803001015],
           dtype='int64', length=2568)


(3131, 10)

In [152]:
blocks.to_csv('CharlotteTrainRowCoords.csv',
          sep=';',
          header=True,
          index=True,
          compression='gzip')

## Vectorize Businesses and Assemble X_train

### Construct Category Corpus with Aliases

In [134]:
lat1000m = -115.161605-(-115.172713)

def makecorpus(df, lat, long) :
    distances = calcDistancesV2(df, lat, long)
    distances = pd.DataFrame(data=distances, index=distances.index, columns=['distances'])
    distances = distances.join(df['alias_cats']) #['alias'] = df.loc[distances.index, 'alias']
    print(distances.shape)
    temp = distances.apply(addproximity, axis=1)
    return ' '.join(temp)

def addproximity(dfrow) :
    w = ('near' if (dfrow.loc['distances'] <= 500) else 'far')
    temp = dfrow.loc['alias_cats'].split(' ')
    temp = list(map((lambda x : (x + '_' + w)), temp))
    return ' '.join(temp)

def calcDistancesV2(multiindexdf, lat, long) :
    minlat = lat - lat1000m
    maxlat = lat + lat1000m
    minlong = long - lat1000m
    maxlong = long + lat1000m
#     multiindexdf = multiindexdf.sort_index(level = [0,1])
    idx = pd.IndexSlice
    df = multiindexdf.loc[idx[minlat:maxlat, minlong:maxlong], ['latitude','longitude']]
    distances = pd.Series(index=df.index)
    for index in df.index :
        try :
            distances[index] = calcDist(lat, long, df.loc[index,'latitude'].values[0], 
                                    df.loc[index,'longitude'].values[0])
        except : # catch very rare attribute error to fill in rest of rows
            distances[index] = 10000 # if calcDist gives error, don't include this business
    distances = distances[distances<1000]
    return distances

def make_X_train(coorddf, businessesdf) :
    for index in coorddf.index :
            coorddf.loc[index,'corpus'] = makecorpus(businessesdf, 
                                            coorddf.loc[index,'latitude'], 
                                            coorddf.loc[index,'longitude'])

In [88]:
df = pd.read_csv('CharlotteBusinessesFinalUpdated.csv',sep=';',compression='gzip',
                 index_col=['lat_index.1','long_index.1'])
df.head()

lat_index  long_index  Unnamed: 0  \
lat_index.1 long_index.1                                      
35.163      -80.849807       35.163  -80.849807         796   
            -80.849670       35.163  -80.849670        4566   
            -80.849541       35.163  -80.849541        1670   
            -80.848420       35.163  -80.848420        3550   
            -80.848419       35.163  -80.848419        3551   

                                                      alias  attributes  \
lat_index.1 long_index.1                                                  
35.163      -80.849807    charlotte-car-transport-charlotte         NaN   
            -80.849670         aaa-pet-services-charlotte-3         NaN   
            -80.849541                paint-craze-charlotte         NaN   
            -80.848420            dy-locksmith-charlotte-13         NaN   
            -80.848419           cielo-apartments-charlotte         NaN   

                                                                 categories  \
lat_index.1 long_index.1                                                      
35.163      -80.849807    [{'alias': 'vehicleshipping', 'title': 'Vehicl...   
            -80.849670    [{'alias': 'vet', 'title': 'Veterinarians'}, {...   
            -80.849541    [{'alias': 'artschools', 'title': 'Art Schools...   
            -80.848420    [{'alias': 'locksmiths', 'title': 'Keys & Lock...   
            -80.848419     [{'alias': 'apartments', 'title': 'Apartments'}]   

                                                                coordinates  \
lat_index.1 long_index.1                                                      
35.163      -80.849807      {'latitude': 35.16333, 'longitude': -80.849807}   
            -80.849670       {'latitude': 35.16303, 'longitude': -80.84967}   
            -80.849541    {'latitude': 35.1628304067365, 'longitude': -8...   
            -80.848420       {'latitude': 35.16336, 'longitude': -80.84842}   
            -80.848419     {'latitude': 35.163356, 'longitude': -80.848419}   

                             distance   latitude  longitude  main_category  \
lat_index.1 long_index.1                                                     
35.163      -80.849807    6926.911525  35.163330 -80.849807     automotive   
            -80.849670    6958.502877  35.163030 -80.849670           pets   
            -80.849541    6979.144871  35.162830 -80.849541  eventservices   
            -80.848420    6909.784556  35.163360 -80.848420   homeservices   
            -80.848419    6910.216948  35.163356 -80.848419   homeservices   

                                             name  price  rating  \
lat_index.1 long_index.1                                           
35.163      -80.849807    Charlotte Car Transport      0     4.0   
            -80.849670           AAA Pet Services      0     4.0   
            -80.849541                Paint Craze      0     5.0   
            -80.848420               DY Locksmith      0     5.0   
            -80.848419           Cielo Apartments      0     3.5   

                          review_count  \
lat_index.1 long_index.1                 
35.163      -80.849807               3   
            -80.849670              10   
            -80.849541               8   
            -80.848420               6   
            -80.848419              11   

                                                           alias_cats  
lat_index.1 long_index.1                                               
35.163      -80.849807                            0_auto 0_automotive  
            -80.849670                         0_groomer 0_vet 0_pets  
            -80.849541    0_eventservices 0_venues 0_education 0_arts  
            -80.848420           0_auto 0_homeservices 0_professional  
            -80.848419                    0_homeservices 0_apartments

In [89]:
df.describe()

,lat_index,long_index,Unnamed: 0,attributes,distance,latitude,longitude,price,rating,review_count
count,4961.000000,4961.000000,4961.000000,0.0,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000
mean,35.214515,-80.839292,3285.536787,NaN,3319.837872,35.214524,-80.839292,0.658335,3.642209,29.721830
std,0.023817,0.029007,1941.795055,NaN,2093.874321,0.023810,0.029007,0.957344,1.283143,88.844471
min,35.163000,-80.916940,0.000000,NaN,11.550093,35.162830,-80.916940,0.000000,1.000000,1.000000
25%,35.200000,-80.856540,1591.000000,NaN,1611.742493,35.200144,-80.856540,0.000000,3.000000,1.000000
50%,35.216000,-80.842493,3245.000000,NaN,2916.189893,35.216026,-80.842493,0.000000,4.000000,4.000000
75%,35.228000,-80.819138,4980.000000,NaN,5358.672228,35.227808,-80.819138,1.000000,5.000000,17.000000
max,35.285000,-80.766733,6613.000000,NaN,6998.487170,35.285230,-80.766733,4.000000,5.000000,1699.000000


In [78]:
blocks = pd.read_csv('CharlotteTrainRowCoords.csv',
          sep=';', compression='gzip', index_col='Unnamed: 0')
blocks.head()

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
371190001001000,907.183167,U,35.230750,-80.848262,371190001001,107319,1073,0,24797,0
371190001001001,731.862622,U,35.229594,-80.846934,371190001001,107319,1073,0,25252,0
371190001001002,759.811243,U,35.228813,-80.847987,371190001001,107319,1073,0,21098,0
371190001001003,847.322318,U,35.228983,-80.848975,371190001001,107319,1073,0,707,0
371190001001004,809.529148,U,35.228088,-80.849034,371190001001,107319,1073,0,21202,0


In [90]:
blocks.describe()

,distance_from_city,latitude,longitude,blockgroup,blockgrouppop,population,landarea,waterarea
count,3131.000000,3131.000000,3131.000000,3.131000e+03,3131.000000,3131.000000,3.131000e+03,3131.000000
mean,3451.190692,35.225142,-80.840285,3.711901e+11,1350.834877,113.513893,6.741900e+04,74.743852
std,1571.385562,0.023721,0.029992,6.972747e+05,670.944925,220.409904,1.151513e+05,819.841334
min,87.675672,35.171854,-80.905814,3.711900e+11,0.000000,0.000000,0.000000e+00,0.000000
25%,2189.543672,35.208149,-80.862491,3.711900e+11,891.000000,0.000000,1.622650e+04,0.000000
50%,3549.843903,35.226113,-80.841560,3.711900e+11,1245.000000,38.000000,2.763300e+04,0.000000
75%,4786.176644,35.243244,-80.817902,3.711900e+11,1758.000000,114.000000,5.825550e+04,0.000000
max,5997.053743,35.277592,-80.775211,3.711998e+11,3687.000000,1578.000000,1.483124e+06,24478.000000


In [91]:
make_X_train(blocks, df)

(4096, 2)
(4133, 2)
(4128, 2)
(4011, 2)
(4026, 2)
(4143, 2)
(4144, 2)
(5475, 2)
(5491, 2)
(5475, 2)
(5471, 2)
(4123, 2)
(4122, 2)
(3992, 2)
(3925, 2)
(5465, 2)
(5468, 2)
(5500, 2)
(5499, 2)
(3961, 2)
(5326, 2)
(2781, 2)
(2589, 2)
(2379, 2)
(708, 2)
(706, 2)
(816, 2)
(2638, 2)
(2752, 2)
(2837, 2)
(2591, 2)
(829, 2)
(3942, 2)
(4101, 2)
(4105, 2)
(4034, 2)
(4121, 2)
(4146, 2)
(4106, 2)
(4140, 2)
(4134, 2)
(5463, 2)
(4156, 2)
(4145, 2)
(3676, 2)
(3811, 2)
(5325, 2)
(5424, 2)
(5448, 2)
(5461, 2)
(5473, 2)
(5485, 2)
(5499, 2)
(5496, 2)
(5470, 2)
(5023, 2)
(5194, 2)
(5151, 2)
(5359, 2)
(5340, 2)
(5450, 2)
(5499, 2)
(5521, 2)
(5517, 2)
(5544, 2)
(5519, 2)
(5510, 2)
(5461, 2)
(5408, 2)
(5536, 2)
(5497, 2)
(4785, 2)
(4688, 2)
(5522, 2)
(4812, 2)
(4781, 2)
(4814, 2)
(5489, 2)
(5485, 2)
(5474, 2)
(5452, 2)
(5453, 2)
(5513, 2)
(5537, 2)
(5546, 2)
(5530, 2)
(5538, 2)
(4185, 2)
(4601, 2)
(2785, 2)
(2763, 2)
(2735, 2)
(4114, 2)
(2738, 2)
(946, 2)
(813, 2)
(824, 2)
(816, 2)
(2640, 2)
(4167, 2)
(4745, 2

(99, 2)
(169, 2)
(203, 2)
(172, 2)
(187, 2)
(150, 2)
(205, 2)
(178, 2)
(167, 2)
(210, 2)
(176, 2)
(220, 2)
(196, 2)
(182, 2)
(406, 2)
(387, 2)
(549, 2)
(513, 2)
(459, 2)
(459, 2)
(168, 2)
(188, 2)
(163, 2)
(139, 2)
(161, 2)
(198, 2)
(212, 2)
(379, 2)
(394, 2)
(447, 2)
(134, 2)
(135, 2)
(118, 2)
(278, 2)
(293, 2)
(310, 2)
(208, 2)
(166, 2)
(203, 2)
(122, 2)
(158, 2)
(267, 2)
(225, 2)
(319, 2)
(194, 2)
(109, 2)
(100, 2)
(465, 2)
(467, 2)
(491, 2)
(436, 2)
(529, 2)
(564, 2)
(498, 2)
(388, 2)
(401, 2)
(383, 2)
(431, 2)
(424, 2)
(432, 2)
(409, 2)
(379, 2)
(333, 2)
(317, 2)
(333, 2)
(407, 2)
(411, 2)
(407, 2)
(443, 2)
(422, 2)
(453, 2)
(505, 2)
(579, 2)
(542, 2)
(575, 2)
(581, 2)
(565, 2)
(588, 2)
(605, 2)
(523, 2)
(592, 2)
(590, 2)
(579, 2)
(553, 2)
(520, 2)
(541, 2)
(471, 2)
(420, 2)
(399, 2)
(354, 2)
(411, 2)
(360, 2)
(419, 2)
(1804, 2)
(1735, 2)
(1761, 2)
(1776, 2)
(1773, 2)
(1778, 2)
(1747, 2)
(429, 2)
(430, 2)
(437, 2)
(471, 2)
(465, 2)
(457, 2)
(1780, 2)
(1789, 2)
(1788, 2)
(1787, 2)


(70, 2)
(68, 2)
(19, 2)
(24, 2)
(20, 2)
(0, 2)
(0, 2)
(26, 2)
(22, 2)
(34, 2)
(31, 2)
(36, 2)
(56, 2)
(60, 2)
(43, 2)
(52, 2)
(61, 2)
(56, 2)
(49, 2)
(47, 2)
(50, 2)
(36, 2)
(51, 2)
(78, 2)
(74, 2)
(73, 2)
(74, 2)
(76, 2)
(77, 2)
(78, 2)
(79, 2)
(73, 2)
(73, 2)
(75, 2)
(65, 2)
(57, 2)
(61, 2)
(77, 2)
(29, 2)
(31, 2)
(40, 2)
(49, 2)
(72, 2)
(52, 2)
(56, 2)
(55, 2)
(73, 2)
(68, 2)
(80, 2)
(97, 2)
(78, 2)
(80, 2)
(80, 2)
(69, 2)
(67, 2)
(68, 2)
(82, 2)
(74, 2)
(38, 2)
(33, 2)
(33, 2)
(30, 2)
(31, 2)
(32, 2)
(30, 2)
(31, 2)
(33, 2)
(15, 2)
(17, 2)
(26, 2)
(0, 2)
(32, 2)
(16, 2)
(26, 2)
(28, 2)
(17, 2)
(21, 2)
(31, 2)
(20, 2)
(44, 2)
(52, 2)
(41, 2)
(53, 2)
(60, 2)
(61, 2)
(49, 2)
(62, 2)
(57, 2)
(0, 2)
(0, 2)
(41, 2)
(34, 2)
(35, 2)
(33, 2)
(39, 2)
(45, 2)
(42, 2)
(43, 2)
(39, 2)
(38, 2)
(37, 2)
(36, 2)
(41, 2)
(33, 2)
(17, 2)
(0, 2)
(0, 2)
(34, 2)
(20, 2)
(37, 2)
(38, 2)
(34, 2)
(36, 2)
(37, 2)
(37, 2)
(37, 2)
(34, 2)
(35, 2)
(31, 2)
(31, 2)
(30, 2)
(43, 2)
(0, 2)
(35, 2)
(19, 2)
(23, 2)


(322, 2)
(639, 2)
(82, 2)
(130, 2)
(130, 2)
(432, 2)
(58, 2)
(58, 2)
(18, 2)
(37, 2)
(48, 2)
(39, 2)
(34, 2)
(25, 2)
(12, 2)
(12, 2)
(282, 2)
(196, 2)
(203, 2)
(163, 2)
(127, 2)
(114, 2)
(46, 2)
(47, 2)
(2, 2)
(2, 2)
(2, 2)
(184, 2)
(184, 2)
(167, 2)
(167, 2)
(178, 2)
(178, 2)
(158, 2)
(158, 2)
(158, 2)
(234, 2)
(177, 2)
(177, 2)
(177, 2)
(177, 2)
(15, 2)
(15, 2)
(15, 2)
(15, 2)
(15, 2)
(15, 2)
(15, 2)
(15, 2)
(15, 2)
(15, 2)
(15, 2)
(15, 2)
(15, 2)
(15, 2)
(15, 2)
(10, 2)
(10, 2)
(10, 2)
(363, 2)
(22, 2)
(22, 2)
(22, 2)
(22, 2)
(22, 2)
(22, 2)
(8, 2)
(60, 2)
(60, 2)
(60, 2)
(60, 2)
(319, 2)
(608, 2)
(608, 2)
(96, 2)
(30, 2)
(30, 2)
(644, 2)
(143, 2)
(143, 2)
(143, 2)
(143, 2)
(143, 2)
(23, 2)
(23, 2)
(11, 2)
(206, 2)
(166, 2)
(167, 2)
(227, 2)
(323, 2)
(239, 2)
(239, 2)
(239, 2)
(428, 2)
(712, 2)
(670, 2)
(670, 2)
(420, 2)
(458, 2)
(493, 2)
(493, 2)
(361, 2)
(47, 2)
(26, 2)
(26, 2)
(26, 2)
(26, 2)
(57, 2)
(66, 2)
(66, 2)
(66, 2)
(66, 2)
(668, 2)
(668, 2)
(668, 2)
(668, 2)
(459, 2)
(27

In [19]:
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2739 entries, 371190001001000 to 1371190053062011
Data columns (total 11 columns):
distance_from_city    2739 non-null float64
urban                 2739 non-null object
latitude              2739 non-null float64
longitude             2739 non-null float64
blockgroup            2739 non-null int64
income                2739 non-null object
blockgrouppop         2739 non-null int64
population            2739 non-null int64
landarea              2739 non-null int64
waterarea             2739 non-null int64
corpus                2739 non-null object
dtypes: float64(3), int64(5), object(3)
memory usage: 336.8+ KB


In [20]:
blocks['corpus'].map((lambda x : len(x.split(' '))))

371190001001000     12870
371190001001001     12714
371190001001002     12714
371190001001003     12556
371190001001004     12646
371190001001005     12768
371190001001006     12755
371190001001007     12944
371190001001008     12949
371190001001009     12984
371190001001010     12793
371190001001011     12706
371190001001012     12737
371190001001013     12105
371190001001014     10608
371190001001015     12747
371190001001016     12974
371190001001017     12960
371190001001018     12937
371190001001019     10700
371190001001020     12112
371190001001021      9718
371190001001022      9482
371190001001023      8615
371190001001024      2342
371190001001025      2198
371190001001026      2483
371190001001027      9645
371190001001028     10060
371190001001029     10205
371190001001030      9430
371190001001031      2513
371190001002000     12621
371190001002001     12902
371190001002002     12839
371190001002003     12693
371190001002004     12931
371190001002005     12964
371190001002

In [21]:
blocks.to_csv('CorpusCharlotteTrainRowCoords.csv',sep=';',compression='gzip')
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2739 entries, 371190001001000 to 1371190053062011
Data columns (total 11 columns):
distance_from_city    2739 non-null float64
urban                 2739 non-null object
latitude              2739 non-null float64
longitude             2739 non-null float64
blockgroup            2739 non-null int64
income                2739 non-null object
blockgrouppop         2739 non-null int64
population            2739 non-null int64
landarea              2739 non-null int64
waterarea             2739 non-null int64
corpus                2739 non-null object
dtypes: float64(3), int64(5), object(3)
memory usage: 336.8+ KB


In [92]:
X_features = joblib.load('X_features.joblib')

vect = CountVectorizer(dtype='uint16',min_df=1, vocabulary=X_features,
                       decode_error='ignore', lowercase=True) 

X_sparse = vect.transform(blocks['corpus'])

In [93]:
dfvect = pd.DataFrame(X_sparse.toarray(),dtype='uint16',columns=X_features)
dfvect.head()

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
0,0,0,36,101,28,142,8,93,0,14,0,3,40,125,1,21,2,44,1,17,0,3,53,151,43,88,5,100,0,2,0,0,19,12,0,2,8,38,2,10,0,38,575,633,0,...,3,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,98,42,72,99,32,71,6,9,0,3,119,47,1,21,2,44,6,12,1,2,173,31,55,76,54,51,1,1,0,0,21,10,2,0,26,24,2,10,38,0,1161,49,0,...,3,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,95,45,71,99,25,77,6,9,0,3,119,48,1,21,2,45,5,13,1,2,173,32,55,78,54,50,1,2,0,0,21,6,2,0,26,20,2,10,38,0,1152,56,0,...,3,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,48,92,58,96,24,76,3,11,0,3,43,124,0,14,2,36,1,9,0,3,58,142,44,66,11,79,1,2,0,0,20,6,2,0,22,23,2,10,0,38,581,625,0,...,3,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,52,88,63,92,25,76,5,10,0,3,44,124,0,14,2,37,5,5,1,3,58,143,51,61,11,78,1,2,0,0,20,6,2,0,22,24,2,10,0,38,577,625,0,...,3,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [94]:
joblib.dump(dfvect, 'X_trainCharlotte.joblib', compress=9)

['X_trainCharlotte.joblib']

In [7]:
y_trainCharlotte = blocks['income']

In [8]:
joblib.dump(y_trainCharlotte, 'y_trainCharlotteUpdated.joblib', compress=9)

['y_trainCharlotteUpdated.joblib']

#### Erroneous Category Mapping in a Few X Train Rows

In [95]:
blocks.tail()

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea,corpus
2371190053061003,5933.042594,U,35.261259,-80.792524,371190053061,17491,554,296,329876,1126,0_hvac_far 0_homeservices_far 0_health_far 0_m...
1371190053062011,5857.990999,U,35.255292,-80.787812,371190053062,13111,1863,254,638762,0,2_hairstylists_far 2_beautysvc_far 2_food_far ...
2371190053062011,5857.990999,U,35.255292,-80.787812,371190053062,13111,1863,254,638762,0,2_hairstylists_far 2_beautysvc_far 2_food_far ...
1371190054012025,5848.553945,U,35.276025,-80.858412,371190054012,21270,2450,136,178717,0,distances alias_cats
1371190054013018,5446.085184,U,35.273665,-80.851228,371190054013,16396,1660,134,214550,51,0_localservices_far 0_religiousorgs_far 0_even...


In [99]:
blocks[blocks['corpus'].str.contains('distances alias_cats')]

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea,corpus
371190012001000,5871.154165,U,35.233146,-80.777016,371190012001,25146,1959,19,44297,0,distances alias_cats
371190012001001,5603.693132,U,35.233074,-80.779974,371190012001,25146,1959,100,57019,0,distances alias_cats
371190012001002,5415.527516,U,35.232882,-80.782030,371190012001,25146,1959,93,88295,0,distances alias_cats
371190012001003,5820.073036,U,35.232013,-80.777391,371190012001,25146,1959,50,59852,0,distances alias_cats
371190012001005,5996.770862,U,35.230319,-80.775211,371190012001,25146,1959,105,48273,0,distances alias_cats
371190012001006,5898.985565,U,35.230631,-80.776327,371190012001,25146,1959,38,26751,0,distances alias_cats
371190012003000,5302.550264,U,35.234448,-80.783624,371190012003,73651,1795,0,13755,0,distances alias_cats
371190012003001,5224.519286,U,35.234732,-80.784567,371190012003,73651,1795,19,11930,0,distances alias_cats
371190012003002,5133.164233,U,35.234842,-80.785620,371190012003,73651,1795,23,14803,0,distances alias_cats
371190012003003,5098.122938,U,35.233600,-80.785715,371190012003,73651,1795,21,14629,0,distances alias_cats


In [104]:
test = blocks[blocks['corpus'].str.contains('distances alias_cats')].copy()
make_X_train(test, df)

(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Empty DataFrame
Columns: [distances, alias_cats]
Index: []
(0, 2)
Emp

In [139]:
# test = pd.DataFrame(data=[[1,'hvac'],[0,'np.nan']], columns=['distances','alias_cats'])
test = pd.DataFrame(data=None, columns=['col1','alias_cats'])

In [140]:
test

,col1,alias_cats


In [141]:
test.apply(addproximity, axis=1)
# having words in the corpus column not in X_features are excluded in count vectorized dfvect

,col1,alias_cats


In [119]:
w = 'near'
temp = dfrow.loc['alias_cats'].split(' ')
temp = list(map((lambda x : (x + '_' + w)), temp))
' '.join(temp)

'_near'